In [ ]:
# copy inside the 'quotation marks' the url address of the ToposText page
url_topostext = ' ........ '

Navigate https://github.com/PerseusDL/canonical-greekLit/tree/master, open View Raw

In [ ]:
# copy inside the 'quotation marks' the url address of the Perseus Github page
url_perseus = ' ......... '

In [ ]:
## assign the name of the work
work = ' ....... '

In [ ]:
pip install -r requirements.txt

In [ ]:
pip install cython

In [ ]:
pip install torch

In [ ]:
import pandas as pd
import re
import csv
from bs4 import BeautifulSoup
import requests
import os
import torch
import logging
import numpy as np
from tqdm import tqdm
from scipy.stats import entropy
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
main_folder = work
subfolders = ["books", "labse_embeddings"]
for subfolder in subfolders:
    os.makedirs(os.path.join(main_folder, subfolder), exist_ok=True)
os.makedirs('data/intermediate/alignments', exist_ok=True)

In [ ]:
pip install flair

In [ ]:
from flair.data import Sentence
from flair.nn import Classifier
from flair.models import SequenceTagger

In [ ]:
tagger = Classifier.load('ner-large')

In [ ]:
access_topostext = requests.get(url_topostext)
soup_topostext = BeautifulSoup(access_topostext.content, 'html.parser')
topostext_page = soup_topostext.find_all('html')

In [ ]:
reference_column = []
book_column = []
token_column = []
flair_ner_column = []
topostext_id_column = []

for html in topostext_page:
    divisions = html.find_all("div", class_="pSelector") 
    for div in divisions:   
        p_tag_id = div.get("id")
        print(p_tag_id)
        if p_tag_id != 'E':
            match = re.match(r"^\d+", p_tag_id)
            if match:
                book = int(match.group())
            paragraph = div.find('p') 
            text = paragraph.get_text()
            sentence = Sentence(text) 
            tagger.predict(sentence)
            
            reference_temp_column = []
            book_temp_column = []
            index_temp_column = []
            token_temp_column = []
            start_pos_temp_column = []
            flair_ner_temp_column = []
            topostext_id_temp_column = []
            
            topostext_id_special_temp_column = []
            start_pos_special_temp_column = []
            
            for i,token in enumerate(sentence):
                token_text = token.text
                start_pos = token.start_position
                
                reference_temp_column.append(p_tag_id)
                book_temp_column.append(book)
                index_temp_column.append(i)
                token_temp_column.append(token_text)
                start_pos_temp_column.append(start_pos)
                flair_ner_temp_column.append('O')
                topostext_id_temp_column.append('-')
            
            for entity in sentence.get_spans('ner'):
                entity_label = entity.labels[0].value
                for i, tok in enumerate(entity):    
                    start_pos_tok = tok.start_position
                    if i == 0:
                        tok_label = 'B-'+str(entity_label)
                    else :
                        tok_label = 'I-'+str(entity_label)
                    for x,y in enumerate(start_pos_temp_column):
                        if int(y) == int(start_pos_tok):
                            flair_ner_temp_column[x] = tok_label
                            
            tags = paragraph.find_all('a')
            for tag in tags:
                if tag.get('about'):
                    topostext_id = tag.get('about') 
                elif tag.get('href'):
                    topostext_id = tag.get('href') 
                topostext_id_special_temp_column.append(topostext_id)  
            
            clean_text=str(paragraph) 
            clean_text = re.sub('<a[^>]*>', 'µ', clean_text) 
            clean_text = re.sub('</a>', '', clean_text) 
            clean_text = re.sub('<p[^>]*>', '', clean_text) 
            clean_text = re.sub('</p>', '', clean_text)
            clean_text = re.sub('<br[^>]*>', '', clean_text) 
            clean_text = re.sub('</br>', '', clean_text)
            clean_text = re.sub('<b[^>]*>', '', clean_text) 
            clean_text = re.sub('</b>', '', clean_text)
            
            list_special_char_seen = 0   
            for i, character in enumerate(clean_text): 
                if character == "µ" :
                    start_pos = int(i-list_special_char_seen)
                    list_special_char_seen += 1 
                    start_pos_special_temp_column.append(start_pos) 
            for i,start_pos_special in enumerate(start_pos_special_temp_column):
                for n,start_pos in enumerate(start_pos_temp_column):
                    if start_pos_special == start_pos:
                        topostext_id_temp_column[n] = topostext_id_special_temp_column[i]
            
            reference_column.extend(reference_temp_column)
            book_column.extend(book_temp_column)
            token_column.extend(token_temp_column)
            flair_ner_column.extend(flair_ner_temp_column)
            topostext_id_column.extend(topostext_id_temp_column)
            
print('Finished!')

In [ ]:
data = {'reference':reference_column, 'book':book_column, 'token':token_column, 'flair_ner':flair_ner_column,'topostext':topostext_id_column}
flairToposText = pd.DataFrame(data) 

In [ ]:
flairToposText['entity_id'] = '-'
count_ent = -1
for i,ent in enumerate(flairToposText['flair_ner']):
    if len(ent) > 1:
        if ent.startswith('B'):
            count_ent = count_ent+1
            flairToposText.loc[i,'entity_id'] = count_ent 
        else : flairToposText.loc[i, 'entity_id'] = count_ent

In [ ]:
entity_ids = [e for e in flairToposText['entity_id'].unique() if isinstance(e, int)]
entity_id_column = []
flair_ner_column = []
topostext_id_column = []

for n in tqdm(range(min(entity_ids), max(entity_ids)+1)):
    entity_id_column.append(n)
    filter_entity = flairToposText[flairToposText['entity_id'] == n]
    filter_entity = filter_entity.reset_index()
    entity_ner = re.findall(r"B-([A-Z]+)", filter_entity['flair_ner'][0])
    flair_ner_column.append(entity_ner[0])
    for i,token in enumerate(filter_entity['token']):
        flag = False
        if filter_entity['topostext'][i] != '-':
            topostext_id_column.append(filter_entity['topostext'][i])
            flag = True
            break        
    if flag == False:
        topostext_id_column.append('-')

UniqueEntities = pd.DataFrame(
{'entity_id': entity_id_column,
'topostext_id': topostext_id_column,
'flair_ner' : flair_ner_column
})

In [ ]:
punctuations = ['.', '!', '?']
sentence_column = []
first_book = min(flairToposText['book'])
last_book = max(flairToposText['book'])
for i1 in range(first_book, last_book+1):
    filter = flairToposText[flairToposText['book'] == i1]
    filter = filter.reset_index()
    count_sentence = 0    
    for i2,token in enumerate(filter['token']):
        sentence_column.append(count_sentence)    
        if token in punctuations: 
            count_sentence = count_sentence + 1
flairToposText['sentence'] = sentence_column

In [ ]:
## here you can save as .csv the annotated English translation
flairToposText.drop(columns=['topostext']).to_csv(main_folder+'/'+work+'_eng.csv', sep= ',')

In [ ]:
for i in range(first_book, last_book+1):
    filter_book = flairToposText[flairToposText['book'] == i]
    name= main_folder+'/books/engbook'+str(i)
    with open(name, "w", encoding='utf-8') as file: 
        for i in range(0, filter_book['sentence'].max()+1):
            filter_rows = filter_book[filter_book['sentence'] == i]
            concatenate_sentence = ' '.join(filter_rows['token'].astype(str))
            file.write(f"{concatenate_sentence}\n")

In [ ]:
access_perseus = requests.get(url_perseus)
soup_perseus = BeautifulSoup(access_perseus.content, 'html.parser')

In [ ]:
chapters = soup_perseus.find_all("div", type="textpart")
reference_column = []
book_column = []
token_column = []
for chapter in chapters:
    reference = chapter.get("n")
    p_tag = chapter.find("p") 
    p_contents = p_tag.get_text()
    words = p_contents.split()
    for i, word in enumerate(words):
        #reference = w_tag.get("n")
        reference_column.append(reference)
        #match = re.match(r"^\d+", reference)
        #if match:
        #  book = int(match.group())
        book = int(reference)
        book_column.append(book)
        #token = w_tag.get_text()
        token = word
        token_column.append(token)

#data = {'reference': reference_column, 'book':book_column, 'token': token_column}
data = {'reference': reference_column, 'book':book_column, 'token': token_column}
GreekText = pd.DataFrame(data)

In [ ]:
sentence_column = []
first_book = min(GreekText['book'])
last_book = max(GreekText['book'])
for i1 in range(first_book, last_book+1):
    filter_book = GreekText[GreekText['book'] == i1]
    filter_book = filter_book.reset_index()
    count_sentence = 0    
    for i2,token in enumerate(filter_book['token']):
        sentence_column.append(count_sentence)    
        if token in punctuations: 
            count_sentence = count_sentence + 1
GreekText['sentence'] = sentence_column
GreekText['index'] = GreekText.index
GreekText['entity_id'] = '-' 

In [ ]:
for i in range(first_book, last_book+1):
    filter_book = GreekText[GreekText['book'] == i]
    name= main_folder+'/books/grcbook'+str(i)
    with open(name, "w", encoding='utf-8') as file: 
        for i in range(0, filter_book['sentence'].max()+1):
            filter_rows = filter_book[filter_book['sentence'] == i]
            concatenate_sentence = ' '.join(filter_rows['token'].astype(str))
            file.write(f"{concatenate_sentence}\n")

In [ ]:
from utils import overlap
from utils.dp_core import *
from setuptools import distutils
from utils import vecalign

In [ ]:
for i in tqdm(range(first_book,last_book+1)): 
    overlap.make_overlap(source = main_folder+'/books/engbook'+str(i), 
                     target = main_folder+'/books/engoverlaps'+str(i)+'.eng', 
                     num_overlaps = 10
                     )
    overlap.make_overlap(source = main_folder+'/books/grcbook'+str(i), 
                     target = main_folder+'/books/grcoverlaps'+str(i)+'.grc', 
                     num_overlaps = 10
                     )

In [ ]:
import sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
renew_labse_files = False
basedir = main_folder+'/books'
lang1 = 'eng'
lang2 = 'grc'
files = os.listdir(basedir)
files = [f for f in files if os.path.isfile(basedir+'/'+f) and f.strip().endswith(lang1)]
all_embeddings = []
for i in files:    
    path = basedir+'/'+i
    print(path)
    with open(path, "r", encoding="utf8") as engfile:
        overlap_sentences_eng = []
        for line in engfile: 
            overlap_sentences_eng.append(line)
    path = path.replace('engoverlaps', 'grcoverlaps')
    path = path.replace('.eng', '.grc')
    print(path)
    with open(path, "r", encoding="utf8") as grcfile:
        overlap_sentences_grc = []
        for line in grcfile:  
            overlap_sentences_grc.append(line)
    j = i.split('.')[0]
    j = j.replace('engoverlaps', '').strip()
    src_embedded_filename = 'overlaps.'+lang1+'.'+j+'.LaBSE.emb'
    tgt_embedded_filename = 'overlaps.'+lang2+'.'+j+'.LaBSE.emb'
    engoverlap_missing = not os.path.exists(main_folder+'/labse_embeddings/overlaps.eng.'+str(j)+'.LaBSE.emb')
    grcoverlap_missing = not os.path.exists(main_folder+'/labse_embeddings/overlaps.grc.'+str(j)+'.LaBSE.emb')
    if renew_labse_files or engoverlap_missing: 
        print(f'making english embeddings: {len(overlap_sentences_eng)}')
        overlaps_embeddings_eng_LaBSE = model.encode(overlap_sentences_eng)
        overlaps_embeddings_eng_LaBSE.tofile(main_folder+'/labse_embeddings/overlaps.eng.'+str(j)+'.LaBSE.emb')
    else:
        print('Skipping creation of '+src_embedded_filename+'. Embeddings exist and cell is in no-ovverride mode.')
    if renew_labse_files or grcoverlap_missing:
        print(f'making greek embeddings: {len(overlap_sentences_grc)}')
        overlaps_embeddings_grc_LaBSE = model.encode(overlap_sentences_grc)
        overlaps_embeddings_grc_LaBSE.tofile(main_folder+'/labse_embeddings/overlaps.grc.'+str(j)+'.LaBSE.emb')
    else:
        print('Skipping creation of '+tgt_embedded_filename+'. Embeddings exist and cell is in no-ovverride mode.')

In [ ]:
dir = os.getcwd()
embeddingsfolder = os.path.join(dir, main_folder,'labse_embeddings')
bookids = [book.split('.')[2] for book in os.listdir(embeddingsfolder) if '.eng.' in book]
for book in tqdm(bookids): 
    i = os.path.join(dir, main_folder, 'books', 'engbook'+book)
    path = os.path.join(dir, main_folder, 'books', 'grcbook'+book)
    src_embedded_filename = main_folder+'/labse_embeddings/overlaps.eng.'+book+'.LaBSE.emb'
    tgt_embedded_filename = main_folder+'/labse_embeddings/overlaps.grc.'+book+'.LaBSE.emb'
    vecalign.vecalign_custom(
            src=i, 
            tgt=path, 
            src_embed=(main_folder+'/books/engoverlaps'+book+'.eng', src_embedded_filename), 
            tgt_embed=(main_folder+'/books/grcoverlaps'+book+'.grc', tgt_embedded_filename) 
        )

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, XLMRobertaModel, XLMRobertaTokenizer, AutoModel, AutoConfig

In [ ]:
model = "UGARIT/grc-alignment"
device=torch.device('cpu')
layer: int=8
distortion: float = 0.0
config = AutoConfig.from_pretrained(model, output_hidden_states=True)
emb_model = AutoModelForMaskedLM.from_pretrained(model, config=config)
emb_model.eval()
emb_model.to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model, use_fast=False)

In [ ]:
def get_embed_list(sent_batch) -> torch.Tensor:
        if emb_model is not None:
            with torch.no_grad():
                if not isinstance(sent_batch[0], str):
                    inputs = tokenizer(sent_batch, is_split_into_words=True, padding=True, truncation=True, return_tensors="pt") ## tokenize the sentence
                else:
                    inputs = tokenizer(sent_batch, is_split_into_words=False, padding=True, truncation=True, return_tensors="pt")
                hidden = emb_model(**inputs.to(device))["hidden_states"]  ## create the embeddings
                if layer >= len(hidden):
                    raise ValueError(f"Specified to take embeddings from layer {layer}, but model has only {len(hidden)} layers.")
                outputs = hidden[layer]
                return outputs[:, 1:-1, :]
        else:
            return None 

In [ ]:
def get_similarity(X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    return (cosine_similarity(X, Y) + 1.0) / 2.0

In [ ]:
def iter_max(sim_matrix: np.ndarray, max_count: int=2) -> np.ndarray:
    alpha_ratio = 0.9
    m, n = sim_matrix.shape
    forward = np.eye(n)[sim_matrix.argmax(axis=1)]  # m x n
    backward = np.eye(m)[sim_matrix.argmax(axis=0)]  # n x m
    inter = forward * backward.transpose()
    if min(m, n) <= 2:
        return inter
    new_inter = np.zeros((m, n))
    count = 1
    while count < max_count:
        mask_x = 1.0 - np.tile(inter.sum(1)[:, np.newaxis], (1, n)).clip(0.0, 1.0)
        mask_y = 1.0 - np.tile(inter.sum(0)[np.newaxis, :], (m, 1)).clip(0.0, 1.0)
        mask = ((alpha_ratio * mask_x) + (alpha_ratio * mask_y)).clip(0.0, 1.0)
        mask_zeros = 1.0 - ((1.0 - mask_x) * (1.0 - mask_y))
        if mask_x.sum() < 1.0 or mask_y.sum() < 1.0:
            mask *= 0.0
            mask_zeros *= 0.0
        new_sim = sim_matrix * mask
        fwd = np.eye(n)[new_sim.argmax(axis=1)] * mask_zeros
        bac = np.eye(m)[new_sim.argmax(axis=0)].transpose() * mask_zeros
        new_inter = fwd * bac
        if np.array_equal(inter + new_inter, inter):
                break
        inter = inter + new_inter
        count += 1
    return inter

In [ ]:
path = r"data/intermediate/alignments/sentence_pairs_alignment"
pattern = r'\[(\d+(?:,\s*\d+)*)\](?:,\s*\[(\d+(?:,\s*\d+)*)\])?.*$'

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [ ]:
for book_n,df in flairToposText.groupby('book'): 
    print(book_n)
    file_name = path+str(book_n)+'.txt' 
    filter_book_eng = df
    filter_book_grc = GreekText[GreekText['book'] == book_n]
    with open(file_name, "r") as file:
        for line in file:
            match1 = re.search(pattern,line) 
            index_eng = match1[1] 
            index_grc = match1[2] 
            if index_eng and index_grc: 
                index_eng = index_eng.split(',')
                index_eng = [int(i) for i in index_eng]
                filter_sentence_eng = filter_book_eng[filter_book_eng['sentence'].isin(index_eng)]
                filter_sentence_eng = filter_sentence_eng.reset_index() 
                src_sent = []
                indexes_to_project = []
                ent_ids = []
                for i, engtoken in enumerate(filter_sentence_eng['token']): 
                    src_sent.append(engtoken) 
                    ent_ids.append(filter_sentence_eng['entity_id'][i])
                    if len(filter_sentence_eng['flair_ner'][i]) > 1:
                        indexes_to_project.append(i)
                if len(indexes_to_project) > 0:        
                    index_grc = index_grc.split(',')
                    index_grc = [int(i) for i in index_grc]
                    filter_sentence_grc = filter_book_grc[filter_book_grc['sentence'].isin(index_grc)]
                    filter_sentence_grc = filter_sentence_grc.reset_index()
                    trg_sent = []
                    indexes_to_update = []
                    for i, grctoken in enumerate(filter_sentence_grc['token']):
                        trg_sent.append(grctoken)  
                        indexes_to_update.append(filter_sentence_grc['index'][i])             
                    l1_tokens = [tokenizer.tokenize(word) for word in src_sent] 
                    l2_tokens = [tokenizer.tokenize(word) for word in trg_sent] 
                    bpe_lists = [[bpe for w in sent for bpe in w] for sent in [l1_tokens, l2_tokens]]
                    l1_b2w_map = []
                    for i, wlist in enumerate(l1_tokens):
                        l1_b2w_map += [i for x in wlist]
                    l2_b2w_map = []
                    for i, wlist in enumerate(l2_tokens):
                        l2_b2w_map += [i for x in wlist]
                    vectors = get_embed_list([src_sent, trg_sent]).cpu().detach().numpy()
                    vectors = [vectors[i, :len(bpe_lists[i])] for i in [0, 1]]
                    all_mats = {} 
                    sim = get_similarity(vectors[0], vectors[1])
                    all_mats["itermax"] = iter_max(sim) 
                    aligns = {} 
                    aligns['itermax'] = set()
                    for i in range(len(vectors[0])):
                        for j in range(len(vectors[1])):
                            if all_mats['itermax'][i, j] > 0:
                                aligns['itermax'].add((l1_b2w_map[i], l2_b2w_map[j]))
                    aligns['itermax'] = sorted(aligns['itermax'])                   
                    for index_to_project in indexes_to_project:
                        projection = 0
                        ent_id = ent_ids[index_to_project]
                        for wordpair in aligns['itermax']: 
                            i_engword = int(wordpair[0]) 
                            if index_to_project == i_engword:
                                i_grcword = int(wordpair[1])
                                GreekText['entity_id'][indexes_to_update[i_grcword]] = ent_id 

print('Finished!')

In [ ]:
topostext_id_column = []
external_id_column = []

requestamount = len(list(enumerate(UniqueEntities.topostext_id.unique())))
for i, topostext_id in tqdm(enumerate(UniqueEntities.topostext_id.unique()), total = requestamount):
    if topostext_id != '-':
        topostext_id_column.append(topostext_id)
        if topostext_id.startswith('/'):
            topostext_id = 'https://topostext.org'+topostext_id
        topostext_page = requests.get(topostext_id)
        soup = BeautifulSoup(topostext_page.content)
        external_link = soup.find('a', href=lambda x: x and 'wikidata.org' in x)
        if external_link:
            external_href = external_link['href']
            external_id_column.append(external_href)
        else: external_id_column.append('-')

UniqueToposTextIds = pd.DataFrame(
{'topostext_id': topostext_id_column,
'external_id': external_id_column
})

In [ ]:
GreekText['flair_ner'] = '-'
GreekText['topostext'] = '-'
GreekText['wikidata'] = '-'

for i,ent_id in enumerate(UniqueEntities['entity_id']):
    filter_entity = GreekText[GreekText['entity_id'] == ent_id]
    filter_entity.reset_index(inplace=True)
    for n,flair_ner in enumerate(filter_entity['flair_ner']):
        index_to_update = filter_entity['index'][n]
        GreekText['topostext'][index_to_update] = UniqueEntities['topostext_id'][i]
        if n == 0:
            GreekText['flair_ner'][index_to_update] = 'B-'+UniqueEntities['flair_ner'][i]
        else : GreekText['flair_ner'][index_to_update] = 'I-'+UniqueEntities['flair_ner'][i]

In [ ]:
for i,topostext_id in enumerate(UniqueToposTextIds['topostext_id']):
    filter_topostext = GreekText[GreekText['topostext'] == topostext_id]
    filter_topostext.reset_index(inplace=True)
    for n,flair_ner in enumerate(filter_topostext['flair_ner']):
        index_to_update = filter_topostext['index'][n]
        GreekText['wikidata'][index_to_update] = UniqueToposTextIds['external_id'][i]

In [ ]:
GreekText.drop(columns=['index']).to_csv(main_folder+'/'+work+'_grc.csv', sep= ',')